In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-dataset/__huggingface_repos__.json
/kaggle/input/csqa-dataset/csqa_single_stage_generated.csv


In [2]:
df = pd.read_csv('/kaggle/input/csqa-dataset/csqa_single_stage_generated.csv')

In [3]:
df.head()

,question,correct_combinations,incorrect_combinations,notes_private
0,The sanctions against the school were a punish...,"['ignore AND/OR undermine', 'diminish AND/OR n...","['enforce AND/OR avoid', 'authoritarian AND/OR...",Correct options merge the idea of disregarding...
1,Sammy wanted to go to where the people were. ...,"['city center AND/OR public park', 'shopping m...","['race track AND/OR apartment', 'the desert AN...",Correct options merge populated areas like cit...
2,To locate a choker not located in a jewelry bo...,"['jewelry store AND jewelry store staff', 'jew...","['neck AND jewelry box', 'jewelry box AND bout...",Correct options merge 'jewelry store' with add...
3,Google Maps and other highway and street GPS s...,['digital navigation tools AND/OR traditional ...,['united states highways AND/OR digital naviga...,Correct options integrate the transition from ...
4,"The fox walked from the city into the forest, ...","['natural habitat AND/OR food sources', 'shelt...","['pretty flowers AND/OR dense forest', 'hen ho...",Correct combinations merge facts about the fox...


In [4]:
!pip install -q transformers accelerate bitsandbytes peft trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

In [6]:
from huggingface_hub import login
login(new_session=False)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")

2025-08-16 02:09:22.723189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755310162.916743      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755310162.975266      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [8]:
import random
import ast
import pandas as pd
import numpy as np


def safe_eval(val):
    if isinstance(val, list):  # already list
        return val
    if pd.isna(val):  # NaN
        return []
    if isinstance(val, str):  # string that looks like a list
        return ast.literal_eval(val)
    return []  # fallback

df['correct_combinations'] = df['correct_combinations'].apply(safe_eval)
df['incorrect_combinations'] = df['incorrect_combinations'].apply(safe_eval)


qa_pairs = []
correct_label_cycle = ['A', 'B', 'C', 'D']
label_index = 0  # To cycle through correct answer positions


for idx, row in df.iterrows():
    if len(row['correct_combinations']) == 0 or len(row['incorrect_combinations']) == 0:
        continue  # skip if missing answers
    
    question = row['question']
    # if len(row['correct_combinations']) > 1:
    #     correct_ans = row['correct_combinations'][1]
    # else:
    correct_ans = row['correct_combinations'][0]
    
    incorrect_list = row['incorrect_combinations']
    if len(incorrect_list) >= 3:
        incorrect_ans = random.sample(incorrect_list, 3)
    else:
        incorrect_ans = random.choices(incorrect_list, k=3)
    
    # We'll fix the correct answer position according to label_index
    labels = ['A', 'B', 'C', 'D']
    correct_label = correct_label_cycle[label_index % 4]
    label_index += 1
    
    # Place correct answer in the position indicated by correct_label
    options = incorrect_ans.copy()
    options.insert(labels.index(correct_label), correct_ans)
    
    qa_pairs.append({
        'question': question,
        'A': options[0],
        'B': options[1],
        'C': options[2],
        'D': options[3],
        'correct_label': correct_label,
        'correct_answer_text': correct_ans
    })

qa_df = pd.DataFrame(qa_pairs)
print(f"Generated {len(qa_df)} QA pairs with balanced correct answer positions.")

qa_df.to_csv('csqa_pairs.csv',index=False)

Generated 5000 QA pairs with balanced correct answer positions.


In [9]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,The sanctions against the school were a punish...,ignore AND/OR undermine,enforce AND/OR avoid,yell at AND/OR diminish,authoritarian AND/OR yell at,A,ignore AND/OR undermine
1,Sammy wanted to go to where the people were. ...,race track AND/OR apartment,city center AND/OR public park,the desert AND/OR roadblock,roadblock AND/OR public park,B,city center AND/OR public park
2,To locate a choker not located in a jewelry bo...,boutique AND jewelry display,jewelry box AND boutique staff,jewelry store AND jewelry store staff,neck AND jewelry box,C,jewelry store AND jewelry store staff
3,Google Maps and other highway and street GPS s...,united states highways AND/OR digital navigati...,countryside routes AND/OR traditional atlases,oceans navigation AND/OR real-time traffic upd...,digital navigation tools AND/OR traditional at...,D,digital navigation tools AND/OR traditional at...
4,"The fox walked from the city into the forest, ...",natural habitat AND/OR food sources,hen house AND/OR natural habitat,storybook AND/OR safety from humans,pretty flowers AND/OR dense forest,A,natural habitat AND/OR food sources


In [10]:
def replace(string):
    return string.replace('AND/OR','AND')

In [11]:
qa_df.A = qa_df.A.apply(replace)
qa_df.B = qa_df.B.apply(replace)
qa_df.C = qa_df.C.apply(replace)
qa_df.D = qa_df.D.apply(replace)
qa_df.correct_answer_text = qa_df.correct_answer_text.apply(replace)

In [12]:
qa_df.head()

,question,A,B,C,D,correct_label,correct_answer_text
0,The sanctions against the school were a punish...,ignore AND undermine,enforce AND avoid,yell at AND diminish,authoritarian AND yell at,A,ignore AND undermine
1,Sammy wanted to go to where the people were. ...,race track AND apartment,city center AND public park,the desert AND roadblock,roadblock AND public park,B,city center AND public park
2,To locate a choker not located in a jewelry bo...,boutique AND jewelry display,jewelry box AND boutique staff,jewelry store AND jewelry store staff,neck AND jewelry box,C,jewelry store AND jewelry store staff
3,Google Maps and other highway and street GPS s...,united states highways AND digital navigation ...,countryside routes AND traditional atlases,oceans navigation AND real-time traffic updates,digital navigation tools AND traditional atlases,D,digital navigation tools AND traditional atlases
4,"The fox walked from the city into the forest, ...",natural habitat AND food sources,hen house AND natural habitat,storybook AND safety from humans,pretty flowers AND dense forest,A,natural habitat AND food sources


In [13]:
def zero_shot_cot_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    # Define the set of valid answer choices for quick lookup
    valid_choices = {'A', 'B', 'C', 'D'}

    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:
            # Construct the prompt with a clear, strict instruction
            prompt = f"""Question:
{str(row['question'])}

Options:
A. {str(row['A'])}
B. {str(row['B'])}
C. {str(row['C'])}
D. {str(row['D'])}

Think step by step in your head to arrive at the correct answer, but do not show your reasoning. The answers migh not be straightforward so understand the question and the options first, think and then answer.
Only output the final single capital letter (A, B, C, or D) on the last line.
Answer:"""



            # 1. TOKENIZE the input prompt string
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=2,
                do_sample=False
            )
            
            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_token_ids = output_tokens[0][inputs['input_ids'].shape[1]:]
            generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
            
            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper()
        })
            
    return results

    

# Usage example
results = zero_shot_cot_qa(model,tokenizer, qa_df)
my_results = pd.DataFrame(results)
my_results.to_csv('csqa_zeroshot_cot_results.csv',index=False)

for res in results[:20]:
    print(f"Q: {res['question']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

Q: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
Predicted answer: A
Correct answer: A
-----
Q: Sammy wanted to go to where the people were.  Where might he go?
Predicted answer: B
Correct answer: B
-----
Q: To locate a choker not located in a jewelry box or boutique where would you go?
Predicted answer: C
Correct answer: C
-----
Q: Google Maps and other highway and street GPS services have replaced what?
Predicted answer: D
Correct answer: D
-----
Q: The fox walked from the city into the forest, what was it looking for?
Predicted answer: A
Correct answer: A
-----
Q: What home entertainment equipment requires cable?
Predicted answer: B
Correct answer: B
-----
Q: The only baggage the woman checked was a drawstring bag, where was she heading with it?
Predicted answer: C
Correct answer: C
-----
Q: The forgotten leftovers had gotten quite old, he found it covered in mold in the back of his what?
Predicted answer: 